# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [7]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [8]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [10]:
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
  # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [12]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [13]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

#enumerate: https://www.runoob.com/python/python-func-enumerate.html

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 群益期貨展望2020全球經濟　通膨議題成關注焦點

▲群益期貨董事長孫天山。（圖／記者余弦妙攝）
記者余弦妙／台北報導
隨著中美貿易戰第一階段協議達成共識，對於今年全球經濟展望，群益期貨預估，美元匯率偏弱，人民幣應不會出現去年大貶狀況，今年也須關注通膨議題。群益期貨董事長孫天山今（8）日表示，美股今年多頭是否會延續、金融市場是否過熱，也是市場關注的焦點，但對於今年市場仍充滿信心。
群益期貨表示，由於避險情緒下降，避險類商品如黃金、日圓、公債走勢疲軟，反觀因為聯準會對於升息的態度不繼續堅持，弱勢美元下讓非美貨幣有所表現，其中英鎊更是在長期低迷中物極必反，首相強森獲得了極高的支持率，此外澳幣等原物料貨幣也相當有戲可看，是2020年的潛在明星球員。
群益期貨研究發展部資深協理許績慶說，中美將在15日簽署第一階段協議，未來或許還會有第二或第三階段協議，但美國總統川普今年還有連任選戰要打，今年也是中國規劃達成建設全面小康社會最終年，中美雙方都有任務要達成，因此預估中美會好好談。
▼群益期貨研究發展部資深協理許績慶。（圖／記者余弦妙攝）
另外，許績慶表示，去年10月起聯準會（Fed）每月開始購買短債，當鈔票越印越多，貨幣越來越不值錢，今年美元匯率恐持續偏弱，人民幣應該不會出現如同去年大幅貶值的情況。
最後，許績慶提醒，通膨議題將會是今年關注焦點，此外全球氣候趨於極端，原物料期貨市場價格可能也會出現明顯變化。
----------------------------------------------------------------------

[1] 群益期貨「新春不打烊」　送中獎率百分百刮刮樂、抽iPhone11

▲群益期貨打造微電影。（圖／記者余弦妙攝）
記者余弦妙／台北報導
群益期貨今（8）日公布「新春不打烊」新春活動，今年祭出高額獎金吸引新舊客戶，在農曆年間交易就有機會獲得8888元獎金，群益期貨更表示，今年更特別加碼抽獎活動最大獎為iPhone11，共3支。
群益期貨表示，這是第4年舉辦新春活動，以往每年都會吸引不少客戶參加，今年祭出高額獎金吸引新舊客戶，在新年期間交易海期商品每滿15口或外匯商品每滿5年，即可

----------------------------------------------------------------------

[7] TBC敲定練台生談授權　年代、非凡、三立斷訊風波結束

----------------------------------------------------------------------

[8] 「LINE TAXI叫車平台」推出機場接送服務　單趟1,050元起再加碼送300元乘車券

▲「LINE TAXI叫車平台」推出機場接送服務，再加碼送300元乘車券。（圖／LINE TAXI提供）
車款方面，LINE TAXI機場接送也提供豐富的選擇，像是有單趟只要1,050元經濟實惠的國產車，也有適合接送貴賓的高級商務車，或是便利全家同行的六人座車，而且還可根據個人需求自由加購服務項目，像是配備兒童安全座椅、增加停靠點等等。
LINE TAXI規劃T粉專屬的「T Rewards」會員計劃，針對「趟次」與「里程數」同時累積，若是趟次多，可以快速提高「T Rewards」會員等級，若是「里程數」多，則可以快速累積「T里程」，並從2020年1月正式開放「T里程」兌換乘車券，像是累積450T，可以兌換500元的乘車券。
▲LINE TAXI提供「每日登入獎勵」讓用戶天天領乘車金。（圖／LINE TAXI提供）
----------------------------------------------------------------------

[9] 三星2019第四季營運利潤下滑34%　但優於分析師預期

▲三星電子去年第四季獲利下滑，但優於分析師預期。（圖／路透）
記者謝仁傑／綜合報導
據華爾街日報報導，最近全球半導體銷售的持續下滑給三星電子（Samsung Electronics Co.）的利潤蒙上陰影，預計2019年第四季的營運利潤下滑34%。
三星電子是全球第一大智慧型手機和儲存晶片生產商，2019年最後一季公司營運利潤預估為7.1兆韓元（合61億美元），低於上年同期的10.8兆韓元。這一預期優於標普全球市場（S&P Global Market Intelligence）調查的分析師預期（6.5兆韓元）。
三星電子預估，去年最後一季營收會與前年同期的59兆韓元基本持平。
分析師稱，2020年5G覆蓋範

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股今（8）日早盤受到美伊衝突影響，一度下跌百點，但盤中隨台積電走揚拉抬，大盤由黑翻紅後於平盤震盪。中午過後再度翻黑並擴大跌勢，終場收在11,817點、下跌63點或0.53%，成交量1495.15億元。
台積電（2330）以平盤329.5元作收；股王大立光（3008）以4785元作收、上漲0.95%；鴻海（2317）表現較弱，收在86.5元、下跌2.92%。
類股漲少跌多，光電、半導體小幅走揚，其他電子、電機、電子零組件、塑膠、生技醫療等類股跌幅超過1%。
半導體類股除了有權王台積電推升之外，敦南（5305）、祥碩（5269）、同欣電（6271）、旺宏（2337）、南亞科（2408）、聯發科（2454）等漲幅皆逾1%；誠創（3536）、立積（4968）則跌逾5%。
光電類股中，面板股友達（2409）、群創（3481）齊揚，光學股玉晶光（3406）上漲逾1%，亞光（3019）、揚明光（3504）下跌。
亞股其他主要市場全數走跌，日經指數、上證指數跌幅超過1%，韓國指數、恆生指數亦維持走跌勢態。
----------------------------------------------------------------------

[19] 群創策略合作美國觸控新創Tanvas　發表新一代觸覺反饋車用顯示器

▲群創策略合作美國觸控新創Tanvas發表新一代觸覺反饋車用顯示器。（圖／群創提供）
記者姚惠茹／台北報導
群創（3481）今（8）日宣布與次世代多點觸控新創公司Tanvas Inc.策略合作，發表15吋觸覺反饋車用顯示器，這款搭載最新黑科技TanvasTouch的觸覺反饋，可在光滑物理表面，觸摸到不同材質的觸感，成為CES 2020車載面板一大亮點。
群創表示，傳統震動和電動觸覺回饋系統不適用於大型、曲面等車用顯示器，需要螢幕移動才能觸控感應，而美國西北大學神經系統與機器人實驗室投入觸控反饋技術長達十多年，2011年分拆成立新創Tanvas Inc.，旗下研究的TanvasTouch不受限於接觸介面，透過軟體定義觸覺，讓指尖感受不再被限制，實現無限範圍的觸控效果，精準感應手指動作。
▲左起為群創荷蘭分公司總經理魯迅、群創光電董事長洪進揚、Tanvas Inc.執行長洛普。（圖／群創提供）
群創荷蘭分公司

圖文／CTWANT
勞退新制開辦已滿15年，勞保局預估今年7月將有6萬名年資滿15年、60歲以上純新制民眾可請領給付。若以現行勞退新制專戶內金額388億7千餘萬元來計算，若選擇一次領取所有給付，平均每人可領60萬元；若選擇按月領取，以國人平均壽命84歲來計算，每月可領約2084元。
據《聯合報》報導，勞保局表示，由於民眾可能中斷年資，所以首批請領的實際人數和金額，仍需至7月才會確定；去年11月已公布的勞退新制基金收益數為2312.4億元，收益率9.96%，今年3月若再分配給所有1,177萬個新制勞退帳戶，平均每個帳戶可多分得約1.9萬元。
勞退新制是2005年開始實行，雇主每月替勞工提撥6％的薪資到個人帳戶內；勞工則可自行選擇是否要再多提撥6％。這些基金由勞動基金運用局負責投資運用，據每年盈虧分配到勞工的帳戶；年資滿15年、年齡滿60歲的勞工可選擇一次領回或依平均餘命分月請領。
▲若選擇一次領取所有給付，平均每人可領60萬元。（圖／pixabay）
勞保局預估今年7月至年底，將有6萬多位符合請領資格的民眾，其中40％為製造業、23％為批發及零售業。勞保局表示，一次給付可靈活運用資金，分月請領則可保障老後生活；不過外界也質疑，若每月領到2084元，相比現行的物價水準、通膨壓力來看，可說是杯水車薪。
據東森新聞訪問民眾表示，一般人應該會選擇月領吧，因為現在平均年齡比較高，不過也抱怨月領太少，「領2千塊要做什麼，還不如一次領60萬比較乾脆。」
勞保局對此表示，勞退新制只上路15年，所以多數勞工累績的年資不夠長，而且部分民眾因長年任職於同一間公司，所以可能也會有舊制勞退年資，可領到退休金。另外，民眾除了勞退新制可請領外，還會有勞保年資，符合者終身可領取勞保年金，目前平均數字為每人1.7萬元。
國民黨總統候選人韓國瑜曾提相關政見，主張低薪青年勞工有自提勞退時，政府再相對應提撥同比率；勞動部則粗估這項計畫每年將多增加300餘億元的支出，必須再行籌措財源。
----------------------------------------------------------------------

[28] 影／美伊衝突促油價飆漲　專家：川普不會讓它維持太久

▲專家指出，美國總統川普的目標是在大選前將油價維持在低位。（圖／路透）
記者王曉敏／綜合報導
美方證實美軍駐伊拉克「阿薩

▲AIA友邦人壽總經理侯文成。（圖／記者楊絡懸攝）
記者楊絡懸／台北報導
根據瑞普萊坊不動產市場研究機構於2019年發表的「財富報告」指出，台灣淨資產（不含住宅）約10億元的超級富豪，人數高達1781人，預估未來5年將以25%幅度成長。看準高資產族群與家族傳承的需求，AIA友邦人壽總經理侯文成表示，台灣富豪人口將是保險「新藍海」，且看好今年的保險市場將更勝去年，民眾可透過美元利率變動型終身壽險，協助資產傳承配置。
侯文成指出，台灣總保費收入高，保額卻低。根據2018年資料，台灣總保費收入達3.5兆元，但壽險投保率僅69%，每人平均保額僅183萬元，其中，保險商品以利變型商品55%最多，其次是投資型商品33%、傳統壽險僅8%。
「台灣超級富豪人數為全球排名第19名，年增4%，優於全球與亞洲平均成長水準。」侯文成表示，台灣有140幾萬家中小企業，20%人口擁有台灣77%資產；台灣市場資金狀況為36兆元，壽險僅3.5兆元保費，「也就是說，高淨值資產人士（HNW）就成為壽險業的藍海市場。」
為了因應此趨勢，AIA友邦人壽新推出「友邦人壽守富人生美元利率變動型終身壽險」，該保單內容是6年期繳費，以美元計價，最高投保金額可達1000萬美元，投保年齡最高可達74歲，可根據被保險人年齡對應不同的免體檢額度，最高可至100萬美元，且提供「豁免保險費」功能，目前宣告利率達3.6%，也享有「回饋分享金」，滿足高資產家族多元資產配置與財富管理的需求。
▲安永聯合會計師事務所執業會計師林志翔整理介紹，近年國際法規趨勢對財富傳承規劃的影響。（圖／記者楊絡懸攝）
此外，安永聯合會計師事務所執業會計師林志翔也表示，國際反洗錢、反避稅趨勢，境外公司於「免稅天堂」逐步訂定經濟實質法案，進行資產規劃的傳承風險及成本也將越來越高；台灣今年起加入實施「共同申報準則」（CRS），預計最快2020年8月起，反避稅措施中的「受控外國公司」（CFC）制度也將上路。
「家族傳承是國安等級的問題。」林志翔說，台灣有140幾萬家企業，其中，中小企業占比98%，而中小企業絕大多數是家族企業，剩下的上市櫃公司，70%也是家族企業（定義為家族實股超過3成、或董事會中家族人數過半，即屬家族企業），因此，家族傳承是否成功，會不會造成家庭紛爭，也屬於國家安全的議題範疇。
林志翔說，如果家庭傳承有問題，產生家族紛爭，也會進而影響台灣就

▲LINE Pay聯手星巴克推出集點換超萌手機支架活動。(圖／品牌提供，下同）
消費中心／綜合報導
▲莎莉躲在馬克杯中俏皮探頭，萌翻粉絲！
這次聯名集點活動推出的「限量手機支架」，除了熊大慵懶地靠在星巴克經典全白咖啡杯旁，超可愛的莎莉泡在杯子裡俏皮探頭，兩種款式萌萌登場。不只圖案萌得嫑嫑，極具質感以及超實用的設計，就是要讓大家邊喝著星巴克熱飲時，還可以展現優雅氣息、從容不迫地邊滑手機。
▲LINE Pay X星巴克限定聯名手機支架共有莎莉、熊大兩款，既實用又可愛。
不管是選萌萌熊大或者療癒莎莉，這兩款手機支架都可讓粉絲在自拍時，不用再擔心手滑摔壞手機。除了平常看影片時超方便，聽完音樂時，手機支架也可化身為耳機繞線器，輕鬆向總是打結的耳機線說掰掰！
這次暖冬優惠一推出，就讓許多粉絲超興奮，除了實用又可愛的熊大、莎莉的手機支架，超值的回饋也讓粉絲紛紛留言「喝一波！」、「這個有心動」、「喝起來呀！」喜愛星巴克的粉絲們，趕緊把握機會好好享受吧！
----------------------------------------------------------------------

[51] 金管會點頭！北富銀一口氣增設8家分行　近年來一次增設最多

▲金管會公布去年11月申設國內分支機構核准名單，其中北富銀增設8家、中信銀1家分行。（圖／資料照）
記者紀佳妘／台北報導
金管會今（7）日公布2019年11月金融機構申請增設國內分支機構核准名單，這次共有2家銀行提出申請設立分支機構，其中一家為台北富邦銀行一口氣申請8家分行，也是近年來增設分行數目最多的銀行，未來北富銀分行將達135家，另外中國信託商業銀行也增設1家。
金管會放寬金融機構得於每年5月及11月分別提出申請，而去年11月共有2家銀行提出新申設分行，經評估整體經濟金融發展情勢、城鄉均衡發展及申請銀行財業務狀況後，銀行局同意北富銀增設8家分行、中信銀增設1家分行，也是近年來增設分行數目最多的銀行，其次為2018年上海商銀增設3家分行。
銀行局表示，北富銀增設2次在桃園市，其餘新竹縣、苗栗縣、台南市、雲林縣、台東縣等區域，按照規定，北富銀須在1年內將8家分行增設完畢，目前北富銀總行加分行共有127家，未來加上增設分行後，可提高到135家。
銀行局指出，每家銀行增設分支機構，每年都會檢視營運方針，而北富銀考量分行經營績

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
鴻海（2317）今（7）日發出聲明，針對有關媒體指稱鴻海因為蘋果而取消在印度馬哈拉施特拉邦（Maharashtra）投資案一事，強調集團與客戶間在印度的設廠之意見不同，為錯誤不實說法，目前集團配合主要客戶在當地生產進度一切順暢，特此澄清。
根據《THE HINDU》報導，鴻海由於和客戶蘋果之間存有分歧意見，因此取消在印度馬哈拉施特拉邦（Maharashtra）的投資案，就連馬哈拉施特拉邦工業部部長德賽（Subhash Desai）都在週一表示，當地與鴻海之間已經取消建廠計畫。
報導提到，鴻海在2015年與印度馬哈拉施特拉邦當地政府簽署合作備忘錄，計劃在五年投資約50億美元（約台幣1515億）設立手機、筆電製造與研發工廠，如今德賽向媒體指出，主要是因為鴻海與蘋果之間的內部分歧，導致計畫無法順利進行。
對此，鴻海發出聲明表示，針對媒體報導，集團與客戶間在印度的設廠之意見不同，為錯誤不實說法，目前集團配合主要客戶在當地生產進度一切順暢，特此澄清。
鴻海強調，集團於全球各地的投資規劃，皆會定期檢視各國政經情勢，並配合客戶的需求調整，為客戶、公司、股東、員工以及合作夥伴爭取最大權益。
----------------------------------------------------------------------

[62] 5G競標第20天達1065.43億元　4回合無人出價趨收歛

----------------------------------------------------------------------

[63] 好成績！去年第4季出口金額達870.6億美元　創歷史單季新高

▲去年12月出口金額，創歷年單月次高，年增率4.0%，優於市場的預期。（圖／記者徐文彬攝）
記者吳靜君／台北報導
財政部統計處今（7）日公布去年12月海關進出口貿易初步統計，去年12月出口金額為295.0億美元，為歷年單月次高，較去年11月增加了3.2%，較前一年同期增加4.0%，優於市場的預期。
去年第4季的出口金額為870.6億美元，創歷史單季新高，年增1.9%。全年出口金額為3293億美元，較前一年減少了1.4%。
---------------------------------------------

▲iOS更新常出現問題。（圖／翻攝自Apple官網）
記者謝仁傑／綜合報導
英國廣播公司BBC報導，iPhone XR無法正常在英國電信商O2的網路運行。根據多則Twitter上的文章，O2電信商用戶每天都遇到iPhone XR多次訊號中斷的問題，無法撥打和接聽電話，以及收發訊信，也無法藉著LTE連接使用應用程式。
O2是西班牙電信英國公司（Telefónica UK Limited）的簡稱，在英國是僅次於英國電信旗下公司EE的第二大電信商。O2的發言人告訴英國廣播公司BBC，它們正在處理這個狀況，解決影響了某些使用iPhone XR‌客戶的暫時性問題」。
O2未說明有多少客戶受到影響，但它們表示重開機可以暫時解決這個問題。
一位受影響用戶表示，O2客服告訴他，雖然目前尚無法得到證實，但可能是Apple的iOS更新導致了這樣的問題；而這位用戶獲得一個月的免費網路資費做為補償。
另一位用戶Jim Jimley告訴BBC，他在12月16日注意到網路有問題，之後，幾乎沒有任何手機網路訊號，它可能每天只有發出一兩次微弱訊號。
----------------------------------------------------------------------

[76] 渣打估2020年美元可能下滑5%　金價多頭氣勢強挑戰1600美元

▲渣打銀行財富管理處負責人陳太齡。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
渣打銀行今（7）日發布2020年全球投資市場展望，預估美元指數2020年有可能會貶值5%，歐元、英鎊有望成美元走弱的最大受益者，人民幣則受到中美貿易戰緊張情勢消退，可望在6.8至7.2區間波動，而日圓則在105至112間震盪，預期不太會有大幅走揚或下滑的情況。
渣打銀行財富管理處負責人陳太齡表示，全球經濟動能逐漸回穩，企業盈餘恢復成長，以及主要央行保持寬鬆的貨幣政策，配合潛在的財政刺激措施等，預期2020年經濟前景樂觀，全球股票表現將有望優於其他資產類別，上半年整體金融市場可望延續漲勢，下半年投資氣氛則受到潛在風險影響，轉趨謹慎看法。
展望今年全球外匯市場，2018年初開始的強勢美元，在今年可能面臨下行風險，陳太齡認為，美國聯準會（Fed）將持續擴張資產負債表，提供美元充沛的流動性，加上2018年升息、2019降息，利差逐漸縮小，也會對美元產生一定壓力，預期

記者姚惠茹／台北報導
鴻海（2317）今（6）日公告財報，去年12月營收5395.07億元，月減10.48%，年減 12.89%；第4季營收1.74兆元，季增25.24%，年減 3.97%；2019年全年營收達5.34兆元，年增 0.89%，連續第3年創新高紀錄。
從四大產品線表現來看，12月單月增幅，網通、伺服器等企業產品表現最佳，其次是連接器等元件及其他產品，再來是智慧手機等消費暨智能產品，最後則是電腦、平板等運算產品。
至於四大產品線表現，第4季單季增幅，以運算產品最強勁，消費暨智能產品居次，再來是企業產品，最後則是元件及其他產品。
最後四大產品線表現，第4季年增幅度，運算產品表現同樣最佳，其次是企業產品，再來是消費暨智能產品，最後則是元件及其他產品。
法人表示，iPhone 11系列新機銷售暢旺，帶動鴻海去年全年營收再創高，今年則可望受惠網通、伺服器等5G產品需求，維持全年營運動能。
----------------------------------------------------------------------

[89] 聽算命師一句話！彰化婦刮中「100萬頭獎」大驚：真的出運了

▲新年刮刮樂買氣旺，許多人都會到投注站試試手氣。（圖／記者李毓康攝）
記者柯沛辰／綜合報導
彰化一名40歲婦人去年底為了求好運，聽算命師的話去彩券行買刮刮樂，沒想到果真幸運刮中100萬元頭獎，讓她當場驚呼「我真的出運了！」
根據台彩公司轉述，開出頭獎的是彰化縣永靖鄉的「永福彩券行」，中獎婦人年約40歲，是這家店的常客，平均每周買2到3次彩券。
婦人向店家透露，她事前特別去請教一位算命師，對方叮囑2019年是她的大運之年，財位在南方，她才來試手氣，沒想到真的中大獎。
婦人說，她聽信算命師的話，心想趕在2019年結束前試試手氣，因此到家中南方的彩券行買刮刮樂，由於當時正好是17點50分，便挑了17和50號的「2020向前行」刮刮樂兩張。
店家表示，當時婦人刮沒多久，突然大叫「哇，這是10萬嗎？」店員立刻上前確認，但2人仔細一看，這才發現刮出的其實是頭獎100萬元，讓婦人高興直呼「我真的出運了啦！」
▲彰化縣永靖鄉「永福彩券行」。（圖／翻攝自GoogleMaps）
►「2020愛妳愛妳」～超殺IG討論爆款來了！
------------------------------